In [1]:
import requests

In [2]:
header = {
  "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
  "Referer": "https://translate.kakao.com/"
}

def translate_to_jp(text):
  data = {
    "queryLanguage": "kr",
    "resultLanguage": "jp",
    "q": text
  }
  k = requests.post("https://translate.kakao.com/translator/translate.json", data=data, headers=header)
  return " ".join([out for array in k.json()["result"]["output"] for out in array])

def translate_to_kr(text):
  data = {
    "queryLanguage": "jp",
    "resultLanguage": "kr",
    "q": text
  }
  k = requests.post("https://translate.kakao.com/translator/translate.json", data=data, headers=header)
  return " ".join([out for array in k.json()["result"]["output"] for out in array])

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Momerio/meigen_generate_Japanese"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [4]:
tokenizer.encode("<s>")

[2]

In [5]:
def generate(text, **kwargs):
  inputs = tokenizer(translate_to_jp(text), add_special_tokens=False, return_tensors="pt")["input_ids"]
  output = model.generate(inputs.cuda(), **kwargs).cpu().numpy()[0]
  return translate_to_kr(tokenizer.decode(output))

In [6]:
한국어입력 = "취업을 잘하기 위해서는 "
generate(한국어입력, max_length=100, top_k=50, top_p=0.95, repetition_penalty=2.0, early_stpping=True)

'일을 잘하기 위해서는 각자의 기술을 습득해야 한다. 왜냐하면 이 기술을 배우는 것을 배우는 것이 더 어려워지기 쉽기 때문이다. 또 신세나 훈련이 부족할 경우를 대비해 학교에 갈 때까지 시간을 늘리도록 해야 하는 등 과제가 있다고 여겨지는 한편 충분한 기술을 익힐 때까지 기다릴 필요가 있다고 느꼈다 익숙한 지도법을 익히면 힘들 수도 있고 ᄏᄏ라고도 하는 말도 있다는 지적이 있자 2021년에는 교육부도 실태파악을 위한 정보공시제도 도입 검토에 들어갔다'